In [1]:
%load_ext autoreload
%autoreload 2

In [94]:
import os
import logging
import torch

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from buyagenda import BigMoneyBuyAgenda
from config import GameConfig 
from constants import BUY
from enums import StartingSplit
from env import DefaultEnvironment
from player import HeuristicPlayer, load_players, PredictorMLPPlayer
from simulations import simulate

In [5]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [3]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

## MLP Simulations ##

In [4]:
model_name = 'r-r-mcts-10k-score'
model_path = os.path.join(model_dir, model_name)
model = torch.load(model_path, map_location='cpu')
model

In [114]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
players = load_players(['MLP', 'BM'], [model_path], map_location='cpu', train=False)
agent = players[0]
env = DefaultEnvironment(config, players, logger=logger)

In [117]:
simulate(env, 1, None)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:root:Player 0, Turn 0: Estate
BUY:root:Player 1, Turn 1: Silver
BUY:root:Player 0, Turn 1: Estate
BUY:root:Player 1, Turn 2: Silver
BUY:root:Player 0, Turn 2: Estate
BUY:root:Player 1, Turn 3: Gold
BUY:root:Player 0, Turn 3: Estate
BUY:root:Player 1, Turn 4: Silver
BUY:root:Player 0, Turn 4: Estate
BUY:root:Player 1, Turn 5: Silver
BUY:root:Player 0, Turn 5: None
BUY:root:Player 1, Turn 6: Gold
BUY:root:Player 0, Turn 6: Estate
BUY:root:Player 1, Turn 7: Gold
BUY:root:Player 0, Turn 7: Copper
BUY:root:Player 1, Turn 8: Gold
BUY:root:Player 0, Turn 8: Estate
BUY:root:Player 1, Turn 9: Silver
BUY:root:Player 0, Turn 9: Estate
BUY:root:Player 1, Turn 10: Silver
BUY:root:Player 0, Turn 10: Copper
BUY:root:Player 1, Turn 11: Province
BUY:root:Player 0, Turn 11: Copper
BUY:root:Player 1, Turn 12: Gold
BUY:root:Player 0, Turn 12: Silver
BUY:root:Player 1, Turn 13: Province
BUY:root:Player 0, Turn 13: Copper
BUY:root:Player 1, Turn 14: Gold
BUY:root:Pla

Let's analyze the neural network outputs for some simple cases.

In [20]:
from state import ReducedStateFeature

In [21]:
skip = ReducedStateFeature.default_sandbox_feature()
copper = ReducedStateFeature.default_sandbox_feature()
silver = ReducedStateFeature.default_sandbox_feature()
estate = ReducedStateFeature.default_sandbox_feature()

In [22]:
copper[0] = 45
copper[7] = 8
copper

tensor([45., 10.,  8.,  8.,  8., 40., 30.,  8.,  0.,  3.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [23]:
silver[5] = 39
silver[7+5] = 1
silver

tensor([46., 10.,  8.,  8.,  8., 39., 30.,  7.,  0.,  3.,  0.,  0.,  1.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [24]:
estate[2] = 7
estate[7+2] = 4
estate

tensor([46., 10.,  7.,  8.,  8., 40., 30.,  7.,  0.,  4.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [54]:
agent.model(copper), agent.model(silver), agent.model(estate)

TypeError: 'LogisticRegression' object is not callable

## Tree Simulations ##

In [6]:
from mcts import GameTree

In [77]:
tree_name = 'default-r-r-mcts'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-mlog'
rollout_path = os.path.join(model_dir, rollout_name)

In [78]:
tree_name = 'r-r-mcts-bm-100k-score-base-default'
rollout_name = 'mlog-10k-base-default'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [79]:
tree = GameTree.load(tree_path, train=False)

In [34]:
tree._root.children[3]

Parent: None | n: 806, v: 208.5, v_bar: 0.259 c: None | Children: [('Copper', '0.000'), ('Estate', '0.000'), ('Duchy', '0.000'), ('Silver', '0.000'), ('Chapel', '0.000'), ('Moat', '0.000'), ('Village', '0.000'), ('Militia', '0.600'), ('MoneyLender', '0.625'), ('Smithy', '0.000'), ('Council Room', '0.667'), ('Laboratory', '0.000'), ('Market', '0.500'), ('Witch', '0.754'), ('None', '0.500')]

In [129]:
# config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
config = GameConfig(split=StartingSplit.Starting34Split, sandbox=False, num_players=2, must_include=default_kingdom)
players = load_players(['UCT', 'DW'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=True)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=True)[0]
# players = [agent, agent]
env = DefaultEnvironment(config, players, logger=logger)

In [118]:
# logger.setLevel(logging.ERROR)
logger.setLevel(BUY)

In [130]:
sim_data = simulate(env, 1, tree)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:Player 0, Turn 0: Silver
BUY:Player 1, Turn 1: Silver
BUY:Player 0, Turn 1: Militia
BUY:Player 1, Turn 2: Silver
BUY:Player 0, Turn 2: Silver
BUY:Player 1, Turn 3: Witch
BUY:Player 0, Turn 3: Silver
BUY:Player 1, Turn 4: None
BUY:Player 0, Turn 4: Silver
BUY:Player 1, Turn 5: Witch
BUY:Player 0, Turn 5: Province
BUY:Player 1, Turn 6: Silver
BUY:Player 0, Turn 6: Silver
BUY:Player 1, Turn 7: Silver
BUY:Player 0, Turn 7: Silver
BUY:Player 1, Turn 8: Gold
BUY:Player 0, Turn 8: Gold
BUY:Player 1, Turn 9: Silver
BUY:Player 0, Turn 9: Province
BUY:Player 1, Turn 10: Province
BUY:Player 0, Turn 10: Gold
BUY:Player 1, Turn 11: Silver
BUY:Player 0, Turn 11: Gold
BUY:Player 1, Turn 12: Silver
BUY:Player 0, Turn 12: Chapel
BUY:Player 1, Turn 13: Silver
BUY:Player 0, Turn 13: Silver
BUY:Player 1, Turn 14: Province
BUY:Player 0, Turn 14: Duchy
BUY:Player 1, Turn 15: Silver
BUY:Player 0, Turn 15: Silver
BUY:Player 1, Turn 16: Gold
BUY:Player 0, Turn 16: Provi

In [93]:
sim_data.player_df

,Turns,Player,Score,Won,Iter
0,17,0,33.0,False,1
1,18,1,37.0,True,1
2,19,0,24.0,False,2
3,19,1,33.0,True,2
4,22,0,36.0,False,3
...,...,...,...,...,...
1995,20,1,32.0,False,998
1996,21,0,36.0,False,999
1997,21,1,38.0,True,999
1998,17,0,30.0,False,1000


In [114]:
data_dir = os.path.join(project_root, 'data')
data_name ='uct-uct-10k-scores'
data_path = os.path.join(data_dir, data_name)

In [115]:
save(data_path, sim_data.player_df)